# Arbitrary State Preparation with 1Q + multi-controlled Rz (Draft)

Goal. Given a normalized vector psi in $C^{2^n}$, synthesize a unitary $U$ such that
$$U |0^n\rangle = \sum_x \psi_x |x\rangle,$$
using only single-qubit gates and multi-controlled $R_z$ (no measurement, no classical bits).

In [1]:
# Environment & Imports
%pip install qiskit numpy
import numpy as np
from typing import Dict, List, Tuple, Sequence

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import RZGate

np.set_printoptions(precision=5, suppress=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


## Problem:

- Input: psi: np.ndarray of size $2^{2^n}$, complex, normalized (up to floating error).
- Output: QuantumCircuit on n qubits (no classical bits) that prepares psi from |0^n>.
- Constraint: Only 1Q gates and multi-controlled Rz. We realize uniformly-controlled Ry by basis-changes plus multiplexed Rz.
- How to Use:
  - prepare_state_circuit(psi: np.ndarray) -> QuantumCircuit

## Implementation Idea

1. Magnitudes: build a binary tree of uniformly-controlled rotations on qubits $$0\dots n-1$$ (MSB->LSB). At level k, for each control prefix, apply an $R_y(
\theta)$ on target k splitting the block's norm.
2. Phases: apply a diagonal via uniformly-controlled $Rz(\phi)$ so that each basis state $|x>$ acquires the desired relative phase $arg(\psi_x)$.
3. Gate set: identify $$R_y(\theta) = S^† H R_z(\theta) H S.$$ Thus we only need local H, S, Sdg, and multi-controlled Rz.

In [2]:
# Step 0: Utilities

def is_power_of_two(m: int) -> bool:
    return (m & (m - 1) == 0) and m > 0

def normalize_state(psi: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(psi)
    if norm == 0:
        raise ValueError("Input vector has zero norm.")
    return psi / norm

def num_qubits_from_len(m: int) -> int:
    if not is_power_of_two(m):
        raise ValueError(f"Length {m} is not a power of two.")
    return m.bit_length() - 1

In [3]:
# Step 1: Magnitudes extraction (theta table for each level/prefix)

def magnitude_angle_table(psi: np.ndarray) -> List[Dict[int, float]]:
    psi = np.asarray(psi, dtype=complex)
    m = len(psi)
    n = num_qubits_from_len(m)
    theta_levels: List[Dict[int,float]] = [dict() for _ in range(n)]
    indices = np.arange(m, dtype=np.int64)
    for k in range(n):
        for p in range(1 << k):
            # Build prefix mask
            mask_prefix = 0
            for j in range(k):
                bit = (p >> (k-1-j)) & 1
                mask_prefix |= (bit << (n-1-j))
            top_mask = ((~0) << (n - k)) & ((1<<n)-1)
            match = (indices & top_mask) == mask_prefix
            idx_block = indices[match]
            left = idx_block[(idx_block & (1 << (n-1-k))) == 0]
            right = idx_block[(idx_block & (1 << (n-1-k))) != 0]
            A = np.linalg.norm(psi[left])
            B = np.linalg.norm(psi[right])
            denom = max(A*A + B*B, 1e-16)**0.5
            if denom == 0:
                theta = 0.0
            else:
                c = A/denom
                c = min(1.0, max(-1.0, c))
                theta = 2*float(np.arccos(c))
            theta_levels[k][p] = theta
    return theta_levels

In [4]:
# Step 2: Phase extraction (phi table for each level/prefix)

def phase_angle_table(psi: np.ndarray) -> List[Dict[int, float]]:
    psi = np.asarray(psi, dtype=complex)
    m = len(psi)
    n = num_qubits_from_len(m)
    phi_levels: List[Dict[int,float]] = [dict() for _ in range(n)]
    indices = np.arange(m, dtype=np.int64)
    for k in range(n):
        for p in range(1 << k):
            mask_prefix = 0
            for j in range(k):
                bit = (p >> (k-1-j)) & 1
                mask_prefix |= (bit << (n-1-j))
            top_mask = ((~0) << (n - k)) & ((1<<n)-1)
            match = (indices & top_mask) == mask_prefix
            idx_block = indices[match]
            left = idx_block[(idx_block & (1 << (n-1-k))) == 0]
            right = idx_block[(idx_block & (1 << (n-1-k))) != 0]
            SL = psi[left].sum()
            SR = psi[right].sum()
            if np.abs(SL) < 1e-15 or np.abs(SR) < 1e-15:
                phi = 0.0
            else:
                phi = float(np.angle(SR) - np.angle(SL))
            phi = (phi + np.pi) % (2*np.pi) - np.pi
            phi_levels[k][p] = phi
    return phi_levels

In [5]:
# Step 3: Uniformly-controlled rotations via multi-controlled Rz

from qiskit.circuit.library import SGate, SdgGate, HGate

def basis_change_for_Ry(qc, q, inverse=False):
    # Ry(theta) = Sdg ; H ; Rz(theta) ; H ; S on target q.
    if not inverse:
        qc.sdg(q); qc.h(q)
    else:
        qc.h(q); qc.s(q)

def apply_pattern_controls(qc, controls: Sequence[int], pattern: int, k: int):
    # Flip controls with desired 0 to convert to all-ones control set.
    for j, c in enumerate(controls):
        desired = (pattern >> (k-1-j)) & 1
        if desired == 0:
            qc.x(c)

def undo_pattern_controls(qc, controls: Sequence[int], pattern: int, k: int):
    for j, c in enumerate(controls):
        desired = (pattern >> (k-1-j)) & 1
        if desired == 0:
            qc.x(c)

def mcrz_on_pattern(qc, controls: Sequence[int], target: int, angle: float):
    num_ctrl = len(controls)
    if num_ctrl == 0:
        qc.rz(angle, target); return
    gate = RZGate(angle).control(num_ctrl)
    qc.append(gate, list(controls) + [target])

def apply_uniformly_controlled_Rz(qc, target: int, controls: Sequence[int], angle_map: Dict[int, float]):
    k = len(controls)
    for pattern, angle in angle_map.items():
        if abs(angle) < 1e-15:
            continue
        apply_pattern_controls(qc, controls, pattern, k)
        mcrz_on_pattern(qc, controls, target, angle)
        undo_pattern_controls(qc, controls, pattern, k)

def apply_uniformly_controlled_Ry(qc, target: int, controls: Sequence[int], angle_map: Dict[int, float]):
    basis_change_for_Ry(qc, target, inverse=False)
    apply_uniformly_controlled_Rz(qc, target, controls, angle_map)
    basis_change_for_Ry(qc, target, inverse=True)

In [6]:
# Step 4: Assemble the full circuit

def prepare_state_circuit(psi: np.ndarray) -> QuantumCircuit:
    psi = np.asarray(psi, dtype=complex).copy()
    norm = np.linalg.norm(psi)
    psi = psi / (norm if norm != 0 else 1.0)
    m = len(psi)
    n = num_qubits_from_len(m)
    qc = QuantumCircuit(n, name="Prep(|psi>)")
    # Magnitudes
    thetas = magnitude_angle_table(psi)
    for k in range(n):
        controls = list(range(k))
        target = k
        apply_uniformly_controlled_Ry(qc, target, controls, thetas[k])
    # Phases
    phis = phase_angle_table(psi)
    for k in range(n):
        controls = list(range(k))
        target = k
        apply_uniformly_controlled_Rz(qc, target, controls, phis[k])
    return qc

In [7]:
# Step 5: Verification

def global_phase_align(vec: np.ndarray, ref: np.ndarray) -> np.ndarray:
    ip = np.vdot(ref, vec)
    if np.abs(ip) < 1e-15:
        return vec
    return vec * (np.conj(ip)/np.abs(ip))

def prep_and_check(psi: np.ndarray, verbose: bool=True):
    n = num_qubits_from_len(len(psi))
    qc = prepare_state_circuit(psi)
    sv0 = Statevector.from_label('0'*n)
    out = sv0.evolve(qc).data
    ref = psi / (np.linalg.norm(psi) or 1.0)
    out_aligned = global_phase_align(out, ref)
    err = np.linalg.norm(out_aligned - ref)
    if verbose:
        print(qc)
        print("||error||_2 =", float(err))
    return float(err), qc

In [9]:
# Step 6: Demos

rng = np.random.default_rng(123)


# n=3 demo
psi3 = rng.normal(size=8) + 1j*rng.normal(size=8)
err3, qc3 = prep_and_check(psi3)

     ┌─────┐┌───┐┌────────────┐┌───┐┌───┐┌───┐              ┌───┐              »
q_0: ┤ Sdg ├┤ H ├┤ Rz(1.5132) ├┤ H ├┤ S ├┤ X ├──────■───────┤ X ├──────■───────»
     ├─────┤├───┤└────────────┘└───┘└───┘└───┘┌─────┴──────┐└───┘┌─────┴──────┐»
q_1: ┤ Sdg ├┤ H ├─────────────────────────────┤ Rz(2.1031) ├─────┤ Rz(1.2874) ├»
     ├─────┤├───┤                             └────────────┘     └────────────┘»
q_2: ┤ Sdg ├┤ H ├──────────────────────────────────────────────────────────────»
     └─────┘└───┘                                                              »
«     ┌───┐                         ┌───┐┌───┐              ┌───┐              »
«q_0: ┤ X ├─────────────────■───────┤ X ├┤ X ├──────■───────┤ X ├──────■───────»
«     ├───┤┌───┐┌───┐       │       ├───┤└───┘      │       ├───┤      │       »
«q_1: ┤ H ├┤ S ├┤ X ├───────■───────┤ X ├───────────■───────┤ X ├──────■───────»
«     └───┘└───┘└───┘┌──────┴──────┐└───┘     ┌─────┴──────┐└───┘┌─────┴──────┐»
«q_2: ───────────────┤ Rz(0.